# 02 — TaskFlow: API MVP (Spring Boot) — Domínio, Contratos e Testes

**Objetivo:** construir a API mínima do **tasks-service** (Fase 0), para depois a colocar atrás do Gateway (Fase 1).

## Domínio (MVP)
**Entidade `Task`**  
Campos: `id`, `title`, `description`, `status{TODO|DOING|DONE}`, `projectId`, `assignee`, `createdAt`, `updatedAt`.

**Regras**
- `title` obrigatório; `status` começa em `TODO`.
- Transições válidas: `TODO → DOING → DONE` (aceitamos retroceder para fins didáticos).


## Contratos da API (MVP)

- `POST /tasks` — cria tarefa  
- `GET /tasks?status=TODO|DOING|DONE&projectId=...` — lista tarefas  
- `PATCH /tasks/{id}/status` — altera estado  
- (Opcional) `POST /projects`, `GET /projects`

### Exemplo de resposta `Task`
```json
{
  "id": "uuid",
  "title": "String",
  "description": "String",
  "status": "TODO|DOING|DONE",
  "projectId": "String",
  "assignee": "String",
  "createdAt": "2025-10-03T10:00:00Z",
  "updatedAt": "2025-10-03T10:00:00Z"
}
```


## Esqueleto de código (Java) — *para referência*

> Estes *snippets* são ilustrativos (colar no teu projeto, não executar aqui).  
> Comentários em inglês para foco técnico.

**Entity**
```java
// Task.java
// Simple JPA entity for MVP
@Entity
public class Task {
  @Id @GeneratedValue private UUID id;
  @Column(nullable = false) private String title;
  private String description;
  @Enumerated(EnumType.STRING) private Status status = Status.TODO;
  private String projectId;
  private String assignee;
  private Instant createdAt = Instant.now();
  private Instant updatedAt = Instant.now();

  public enum Status { TODO, DOING, DONE }

  @PreUpdate
  public void touch() { this.updatedAt = Instant.now(); }
}
```

**Repository**
```java
// TaskRepository.java
public interface TaskRepository extends JpaRepository<Task, UUID> {
  List<Task> findByStatusAndProjectId(Task.Status status, String projectId);
  List<Task> findByStatus(Task.Status status);
  List<Task> findByProjectId(String projectId);
}
```

**Controller (trechos)**
```java
// TaskController.java
@RestController
@RequestMapping("/tasks")
public class TaskController {
  private final TaskRepository repo;

  public TaskController(TaskRepository repo) { this.repo = repo; }

  @GetMapping
  public List<Task> list(@RequestParam(required = false) Task.Status status,
                         @RequestParam(required = false) String projectId) {
    if (status != null && projectId != null) return repo.findByStatusAndProjectId(status, projectId);
    if (status != null) return repo.findByStatus(status);
    if (projectId != null) return repo.findByProjectId(projectId);
    return repo.findAll();
  }

  @PostMapping
  public ResponseEntity<Task> create(@RequestBody Task body) {
    if (body.getTitle() == null || body.getTitle().isBlank())
      return ResponseEntity.badRequest().build();
    return ResponseEntity.status(HttpStatus.CREATED).body(repo.save(body));
  }

  @PatchMapping("/{id}/status")
  public ResponseEntity<Task> update(@PathVariable UUID id, @RequestBody Map<String, String> payload) {
    return repo.findById(id).map(t -> {
      var s = Task.Status.valueOf(payload.getOrDefault("status","TODO"));
      t.setStatus(s);
      return ResponseEntity.ok(repo.save(t));
    }).orElse(ResponseEntity.notFound().build());
  }
}
```


## Testes por *curl* (local)

> **Direto ao serviço (Fase 0):** porta `8081`

```bash
# Listar
curl -s http://localhost:8081/tasks | jq

# Criar
curl -s -H "Content-Type: application/json" \
  -d '{"title":"Primeira","description":"Demo","projectId":"P1","assignee":"Ana"}' \
  http://localhost:8081/tasks | jq

# Atualizar estado (substituir <ID>)
curl -s -X PATCH -H "Content-Type: application/json" \
  -d '{"status":"DOING"}' \
  http://localhost:8081/tasks/<ID>/status | jq
```
